In [1]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier
from scipy import sparse

In [2]:
train = pd.read_csv('data/mnist_train.csv')
test = pd.read_csv('data/mnist_test.csv')

In [3]:
train.shape, test.shape

((60000, 785), (10000, 785))

In [4]:
train.dropna(inplace=True)
test.dropna(inplace=True)
train.shape, test.shape

((60000, 785), (10000, 785))

In [5]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = train.drop(columns=['label']).values
y_train = train['label'].values
X_test = test.drop(columns=['label']).values
y_test = test['label'].values

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [9]:
clf = SGDClassifier(loss='hinge',
                    penalty='l2',
                    max_iter=500,
                    tol=1e-3,)

In [10]:
%time model = clf.fit(X_train, y_train)

Wall time: 2min 36s


In [11]:
sparse_clf = SGDClassifier(loss='hinge',
                            penalty='l2',
                            max_iter=500,
                            tol=1e-3,)

In [12]:
X_train_sparse = sparse.coo_matrix(X_train)

In [13]:
%time sparse_model = sparse_clf.fit(X_train_sparse, y_train).sparsify()

Wall time: 58 s


In [14]:
%time y_pred = model.predict(X_test)

Wall time: 236 ms


In [15]:
%time X_test_sparse = sparse.coo_matrix(X_test)

Wall time: 126 ms


In [16]:
%time y_pred_sparse = sparse_model.predict(X_test_sparse)

Wall time: 99.8 ms


In [17]:
model.score(X_test, y_test)

0.8794

In [18]:
sparse_model.score(X_test_sparse, y_test)

0.8779

In [20]:
import sys
print(f'Size of regular model: {sys.getsizeof(model)}')
print(f'Size of sparse model: {sys.getsizeof(sparse_model)}')
print(f'Size of regular coefficients: {sys.getsizeof(model.coef_)}')
print(f'Size of sparse coefficients: {sys.getsizeof(sparse_model.coef_)}')

Size of regular model: 56
Size of sparse model: 56
Size of regular coefficients: 62840
Size of sparse coefficients: 56


In [21]:
!ipython profile create

[ProfileCreate] Generating default config file: 'C:\\Users\\Safiuddin\\.ipython\\profile_default\\ipython_config.py'
[ProfileCreate] Generating default config file: 'C:\\Users\\Safiuddin\\.ipython\\profile_default\\ipython_kernel_config.py'


In [22]:
%load_ext memory_profiler

In [23]:
import os
print(f'PID: {os.getpid()}')

PID: 20724


In [24]:
%%memit

with sklearn.config_context(assume_finite=True,
                            working_memory=2):  # Limit working memory to 2MB
    spar_model = clf.fit(X_train_sparse, y_train).sparsify()

peak memory: 561.73 MiB, increment: 155.79 MiB
